In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [6]:
import torch
import torch.nn as nn

In [9]:
class GCN(nn.Module):
    
    def __init__(self, input_dim, h1, output_dim):
        super(GCN, self).__init__()
        self.input_dim = input_dim
        self.h1 = h1
        self.output_dim = output_dim
        self.W1 = nn.Linear(self.input_dim, self.h1)
        self.W2 = nn.Linear(self.h1, self.output_dim)

In [10]:
plt.style.use('seaborn-notebook')

In [11]:
from src.test import *

In [12]:
node_labels = pd.read_csv('datasets/citeseer/citeseer.node_labels', header=None, names=['id', 'label'])

In [13]:
edges = pd.read_csv('datasets/citeseer/citeseer.edges', header=None, names=['e1', 'e2', 'weight'])

In [14]:
G = nx.Graph()
for _, row in node_labels.iterrows():
    G.add_node(row.id, label=row.label)
G.add_edges_from(edges.loc[:, ['e1', 'e2']])